In [29]:
import requests

API_BASE = "http://localhost:8000"
session_id = "test-full-feedback-001"

# 1️⃣ /feedback: 코드 피드백 생성
feedback_req = {
    "title": "반복문 개선",
    "description": "리스트 순회",
    "input_rule": "정수 리스트가 주어진다",
    "output_rule": "모든 요소를 출력한다",
    "sample_input": "1 2 3",
    "sample_output": "1\n2\n3",
    "code_language": "python",
    "code": "for i in range(len(arr)):\n    print(arr[i])"
}

response = requests.post(f"{API_BASE}/api/ai/v2/feedback", json=feedback_req)
assert response.status_code == 200
feedback_data = response.json()

print("✅ 피드백 생성 결과:")
print(feedback_data)

✅ 피드백 생성 결과:
{'title': '반복문 개선', 'good': ['코드 구조가 간결합니다.', '인덱스를 직접 사용하여 요소에 접근합니다.'], 'bad': ['변수 이름 `arr`이 명확하지 않습니다.', '리스트의 길이를 미리 계산하는 것은 불필요합니다.'], 'improved_code': 'for element in arr:\n    print(element)'}


In [30]:
import requests

API_BASE = "http://localhost:8000"
session_id = "demo-feedback-session-001"
history = []

# 1️⃣ 코드 피드백 생성
def start_feedback(code: str, title="제목 없음"):
    feedback_req = {
        "title": title,
        "description": "리스트 순회",
        "input_rule": "정수 리스트가 주어진다",
        "output_rule": "모든 요소를 출력한다",
        "sample_input": "1 2 3",
        "sample_output": "1\n2\n3",
        "code_language": "python",
        "code": code
    }
    r = requests.post(f"{API_BASE}/api/ai/v2/feedback", json=feedback_req)
    r.raise_for_status()
    res = r.json()
    print("✅ 첫 피드백:")
    print("🎯 제목:", res["title"])
    print("👍 잘한 점:", res["good"])
    print("⚠️ 아쉬운 점:", res["bad"])
    print("💡 개선 코드:\n", res["improved_code"])
    history.append({"role": "assistant", "content": str(res)})
    return res

# 2️⃣ 챗봇 대화
def chat_feedback(user_message: str):
    history.append({"role": "user", "content": user_message})
    r = requests.post(f"{API_BASE}/api/ai/v2/feedback/chat", json={
        "session_id": session_id,
        "messages": history
    })
    r.raise_for_status()
    res = r.json()["answer"]
    print("🤖 AI:", res)
    history.append({"role": "assistant", "content": res})
    return res


In [31]:
res = start_feedback("for i in range(len(arr)):\n    print(arr[i])")
chat_feedback("이 코드를 더 파이썬스럽게 바꿀 수 있을까?")
chat_feedback("그럼 list comprehension으로 바꾸면 어떻게 돼?")

✅ 첫 피드백:
🎯 제목: 제목 없음
👍 잘한 점: ['코드 구조가 간결합니다.', '입력 처리를 적절히 했습니다.']
⚠️ 아쉬운 점: ['인덱스를 사용하여 직접 접근하는 것은 권장되지 않습니다.', '변수 이름이 명확하지 않습니다.']
💡 개선 코드:
 arr = [int(x) for x in input().split()]
for element in arr:
    print(element)
🤖 AI: 네, 이 코드를 더 파이썬스럽게 바꿀 수 있습니다. 다음과 같이 수정할 수 있습니다:

```python
numbers = list(map(int, input().split()))
for number in numbers:
    print(number)
```

이렇게 하면 `map` 함수와 `list` 함수를 사용하여 입력을 한 줄로 받고 리스트로 변환할 수 있으며, 변수명도 더 명확해졌습니다.
🤖 AI: 물론입니다! `list comprehension`을 사용하면 다음과 같이 코드를 단축시킬 수 있습니다:

```python
[print(int(x)) for x in input().split()]
```

이 코드는 한 줄로 모든 작업을 수행하며, 가독성이 높아집니다.


'물론입니다! `list comprehension`을 사용하면 다음과 같이 코드를 단축시킬 수 있습니다:\n\n```python\n[print(int(x)) for x in input().split()]\n```\n\n이 코드는 한 줄로 모든 작업을 수행하며, 가독성이 높아집니다.'

In [36]:
import requests

API_BASE = "http://localhost:8000"
session_id = "interview-session-001"
messages = []

# 1️⃣ 면접 시작
def start_interview():
    req = {
        "title": "반복문 출력 문제",
        "number": 1,
        "description": "정수 리스트를 순회하며 값을 출력하는 문제입니다.",
        "input_rule": "정수 리스트가 주어진다",
        "output_rule": "각 줄마다 하나씩 정수를 출력한다",
        "sample_input": "1 2 3",
        "sample_output": "1\n2\n3",
        "code": "for i in range(len(arr)):\n    print(arr[i])",
        "code_language": "python"
    }
    r = requests.post(f"{API_BASE}/api/ai/v2/interview/start", json=req)
    r.raise_for_status()
    q = r.json()["question"]
    print("🎤 첫 질문:", q)
    messages.append({"role": "assistant", "content": q})

# 2️⃣ 후속 질문 요청
def followup(answer):
    messages.append({"role": "user", "content": answer})
    r = requests.post(f"{API_BASE}/api/ai/v2/interview/answer", json={
        "session_id": session_id,
        "messages": messages
    })
    r.raise_for_status()
    q = r.json()["question"]
    print("🎯 후속 질문:", q)
    messages.append({"role": "assistant", "content": q})

# 3️⃣ 인터뷰 종료 후 총평
def end_interview():
    r = requests.post(f"{API_BASE}/api/ai/v2/interview/end", json={
        "session_id": session_id,
        "messages": messages
    })
    r.raise_for_status()
    res = r.json()
    print("✅ 총평:")
    print("👍 잘한 점:", res["review"]["good"])
    print("⚠️ 개선할 점:", res["review"]["bad"])
    print("💡 개선 방향:", res["review"]["improvement"])


In [37]:
start_interview()
followup("range를 쓰면 성능상 장점이 있을까요?")
followup("enumerate로 바꾸면 코드가 더 간단하지 않을까요?")
end_interview()


🎤 첫 질문: How would you optimize this code to improve its time complexity?
🎯 후속 질문: 네, range 함수를 사용하면 일반적인 for 루프보다 더 효율적일 수 있습니다. 이는 range 함수가 내부적으로 반복을 처리하기 때문에, 일반적인 for 루프보다 적은 메모리와 시간을 사용할 수 있기 때문입니다. 하지만, 특정 상황에서는 일반적인 for 루프가 더 효과적일 수도 있습니다. 따라서, 사용하는 상황에 따라 적절한 방법을 선택하는 것이 중요합니다.
🎯 후속 질문: 네, enumerate 함수를 사용하면 인덱스와 해당 값을 동시에 접근할 수 있어 코드가 더 간결해질 수 있습니다. 예를 들어:

```
for i in range(len(my_list)):
    print(i, my_list[i])
```

위의 코드를 다음과 같이 변경할 수 있습니다:

```
for i, value in enumerate(my_list):
    print(i, value)
```

이렇게 하면 코드가 더 간결해지고 가독성이 좋아집니다. 하지만, enumerate 함수도 내부적으로 반복을 처리하므로, 일반적인 for 루프보다 적은 메모리와 시간을 사용할 수 있습니다. 따라서, 사용하는 상황에 따라 적절한 방법을 선택하는 것이 중요합니다.
✅ 총평:
👍 잘한 점: ["The candidate demonstrated a good understanding of Python's built-in functions like `range` and `enumerate`, which can lead to more efficient and readable code.", 'The candidate was able to explain the trade-offs between using `range` and `enumerate` based on specific use cases.']
⚠️ 개선할 점: ['The candidate did no

In [34]:
import requests

API_BASE = "http://localhost:8000"

# 1️⃣ 솔루션 요청
def request_solution():
    req = {
        "problemNumber": 14502,
        "title": "연구소",
        "description": (
            "인체에 치명적인 바이러스를 연구하던 연구소에서 바이러스가 유출되었습니다. "
            "바이러스는 상하좌우로 인접한 빈 칸으로 퍼져나갈 수 있으며, 연구소의 빈 칸 중 "
            "세 곳에 벽을 세워 바이러스의 확산을 막으려 합니다.\n\n"
            "연구소는 N×M 크기의 격자로 표현되며, 각 칸은 빈 칸(0), 벽(1), 바이러스(2) 중 하나입니다. "
            "벽을 정확히 3개 세운 뒤 바이러스가 퍼질 수 없는 안전 영역(0으로 남아 있는 칸)의 최대 크기를 구하십시오."
        ),
        "input": (
            "첫째 줄에 지도의 세로 크기 N과 가로 크기 M이 주어집니다. (3 ≤ N, M ≤ 8)\n"
            "둘째 줄부터 N개의 줄에 연구소의 지도 정보가 주어지며, 각 줄에 M개의 정수가 빈 칸(0), 벽(1), 바이러스(2) 중 하나로 주어집니다."
        ),
        "output": "바이러스가 퍼진 이후 안전 영역의 최대 크기를 출력합니다.",
        "inputExample": (
            "7 7\n"
            "2 0 0 0 1 1 0\n"
            "0 0 1 0 1 2 0\n"
            "0 1 1 0 1 0 0\n"
            "0 1 0 0 0 0 0\n"
            "0 0 0 0 0 1 1\n"
            "0 1 0 0 0 0 0\n"
            "0 1 0 0 0 0 0"
        ),
        "outputExample": "27"
    }


    r = requests.post(f"{API_BASE}/api/ai/v1/solution", json=req)
    r.raise_for_status()
    res = r.json()
    print("✅ 솔루션 생성 결과")
    print("📌 문제 개요:", res["problemCheck"]["problemDescription"])
    print("🧠 알고리즘 분류:", res["problemCheck"]["algorithm"])
    print("💡 해결 전략:\n", res["problemSolving"])
    print("🐍 Python 코드:\n", res["solutionCode"]["python"])
    print("💻 C++ 코드:\n", res["solutionCode"]["cpp"])
    print("📝 Java 코드:\n", res["solutionCode"]["java"])

    return res

In [35]:
request_solution()

HTTPError: 422 Client Error: Unprocessable Content for url: http://localhost:8000/api/ai/v1/solution